In [25]:
import numpy as np

import cv2

In [26]:
h,w = 100, 200

start = (10, h-10)
end = (w-10, 10)
parent_child = [(0, start)]



board = np.zeros((h, w, 3),dtype=np.uint8)
board_buffer = board.copy()
board_buffer = cv2.cvtColor(board_buffer, cv2.COLOR_BGR2GRAY)

circle_center = (int((w/2)+10), int(h/2))
circle_radius = (int((h/2)-20))

board = cv2.circle(board, circle_center, circle_radius, (255, 0, 0), -1)

buffer = 2
board_buffer = cv2.circle(board_buffer, circle_center, circle_radius + buffer, (255), 2)
board_buffer = cv2.rectangle(board_buffer, (0,0), (w,h), 255, buffer)
locations = np.where(board_buffer == (255))
boundary_xy = set(zip(locations[1], locations[0]))

def move_up(index):
    parent_xy = parent_child[index][1]
    child_xy = (parent_xy[0],parent_xy[1]-1)
    if child_xy not in boundary_xy:
        boundary_xy.add(child_xy)
        parent_child.append((index,child_xy))

def move_right(index):
    parent_xy = parent_child[index][1]
    child_xy = (parent_xy[0]+1, parent_xy[1])
    if child_xy not in boundary_xy:
        boundary_xy.add(child_xy)
        parent_child.append((index, child_xy))

def move_down(index):
    parent_xy = parent_child[index][1]
    child_xy = (parent_xy[0], parent_xy[1]+1)
    if child_xy not in boundary_xy:
        boundary_xy.add(child_xy)
        parent_child.append((index, child_xy))

def move_left(index):
    parent_xy = parent_child[index][1]
    child_xy = (parent_xy[0]-1, parent_xy[1])
    if child_xy not in boundary_xy:
        boundary_xy.add(child_xy)
        parent_child.append((index, child_xy))

In [27]:
i = 0
while end not in boundary_xy:
    for item in [move_up, move_down, move_left, move_right]:
        item(i)
    i += 1




In [28]:
path = [parent_child[-1]]
while path[-1][0] > 0:
    parent_idx = path[-1][0]
    path.append(parent_child[parent_idx])

path.reverse()

In [29]:
rainbow = [(0,0,255)]

# spacing = int(1/(255/(len(parent_child))))
for i in range(0,len(parent_child)):
    ratio = i / (len(parent_child))

    if ratio < 1/3:
        B= 255
        G = int(255*(ratio*3))
        R = 0
    elif ratio < 2/3:
        B = int(255*(2-3*ratio))
        G = 255
        R = int(255*(3*ratio-1))
    else:
        B = 0
        G = int(255*(3-3*ratio))
        R = 255
    rainbow.append((B,G,R))

    

In [30]:
frames = []
board_animation = board.copy()


for item in parent_child:
    b,g,r = rainbow.pop()
    rc = (item[1][1], item[1][0])
    board_animation[rc] = b,g,r
    frame = board_animation.copy()
    cv2.imshow('animation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

for item in path:
    rc = (item[1][1], item[1][0])
    board_animation[rc] = 255,255,255
    frame = board_animation.copy()
    cv2.imshow('animation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.waitKey(0)
cv2.destroyAllWindows()